In [ ]:
from nonstationary_demo import (
    AgentParams, TaskSpec, make_correct_pairs_from_reward_probs,
    simulate_reversal_session,
    plot_learning_curves, plot_reversal_aligned
)

states = ["6kHz", "10kHz"]
actions = ["L", "R", "N"]

reward_probs = {
    ("6kHz", "L"): 0.9, ("6kHz", "R"): 0.1, ("6kHz", "N"): 0.0,
    ("10kHz","L"): 0.1, ("10kHz","R"): 0.9, ("10kHz","N"): 0.0,
}
correct_pairs = make_correct_pairs_from_reward_probs(reward_probs)

task = TaskSpec(state_space=states, action_space=actions, reward_probs=reward_probs, correct_pairs=correct_pairs)
agent = AgentParams(alpha=0.2, beta=4.0, bias_L=0.0, bias_R=0.0, bias_N=-1.0)

df = simulate_reversal_session(
    seed=0,
    n_trials=2000,
    agent=agent,
    task=task,
    reversal_countdown=250,
    perf_window=20,
    perf_threshold=19,
)

fig1 = plot_learning_curves(df, window=50)
fig2 = plot_reversal_aligned(df, window=50, before=150, after=150)
